this script will create a pandas dataframe (like a table)

pull data from the detroit mural website, one page at a time(20 murals/page) and add to the dataframe

then export to CSV

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np

In [50]:
#the website we are collecting data from 
page_link = "https://detroitmopa.org/artworks/tag/Murals"
#structure of the database
df =pd.DataFrame(columns = ['muralName', 'artistName', 'address', 'year','imgLink'])


In [77]:
def MuralScrape(page_link, df):
    #function that takes a link and dataframe -like an excel spreadsheet/table -
    #and outputs a dataframe
    page_response = requests.get(page_link, timeout=5)
    #beautiful soup is a useful tool for reading and parsing html
    page_content = BeautifulSoup(page_response.content, "html.parser")
    #finds all html blocks with a certain class name
    chain = [x.find_all('img') for x in page_content.find_all(class_="BlogList-item-image")]
    
    #initialize list to store data
    imagelist = []
    muralnamelist= []
    
    for i in range(len(chain)):
        string = chain[i][0]
        #use regex to get only relevant info
        result = re.search('(?:)data-image="([^"]+)', string.prettify())
        if result:
            imagelist.append(result.group(1))
        else:
            imagelist.append("")
        muralname = re.search('(?<=img alt=")(.*)(?=" data-image=")', string.prettify())
        if muralname:
            muralnamelist.append(muralname.group(1))
        else:
            muralnamelist.append("")
    
    #check to make sure data is added to list 
    print(muralnamelist)
    print(imagelist)
    excerpt = [x.find_all('p') for x in page_content.find_all(class_ = "BlogList-item-excerpt")]
    
    addresslist = []
    artistnamelist= []
    yearlist=[]
    
    for i in range(len(excerpt)):
        string = excerpt[i][0]
    
        name = re.search('(?<=Artist:)([\s\S]*)(?=Year)', string.text)
        if name:
            artistnamelist.append(name.group(1).replace(u'\xa0', u' ').strip())
        else:
            artistnamelist.append("")
        
        year = re.search('(?<=Year: )(.*)(?=Address)', string.text.replace(u'\xa0', u' '))
        if year:
            yearlist.append(year.group(1))
        else:
            yearlist.append("")
        address = re.search('(?<=Address: )([\S\s]*)', string.text.replace(u'\xa0', u' '))
        if address:
        
            addresslist.append(address.group(1))
        else:
            addresslist.append("")
    
    #adds data from lists to dataframe
    for i in range(len(addresslist)):
        newlist = [muralnamelist[i],artistnamelist[i],addresslist[i], yearlist[i], imagelist[i]]
        df = df.append(pd.Series(newlist,index=['muralName', 'artistName', 'address', 'year','imgLink']), ignore_index=True)
    return df
    

In [51]:
#call function
df = MuralScrape(page_link, df)

In [52]:
#show first 5 entries in dataframe
df.head()

,muralName,artistName,address,year,imgLink
0,DETROIT MURAL #0674,Pat Perry,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
1,DETROIT MURAL #0694,Tony Whlgn,"19031 Grand River Ave, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
2,DETROIT MURAL #0693,Ledania,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
3,DETROIT MURAL #0690,FFTY,"8801 Oakland Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
4,DETROIT MURAL #0689,Laura Finlay,"4126 Third Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...


In [54]:
# scrape next page 
page_link = "https://detroitmopa.org/artworks?offset=1528831925675&tag=Murals"
df = df.append(MuralScrape(page_link, df), ignore_index=True)
df

,muralName,artistName,address,year,imgLink
0,DETROIT MURAL #0674,Pat Perry,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
1,DETROIT MURAL #0694,Tony Whlgn,"19031 Grand River Ave, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
2,DETROIT MURAL #0693,Ledania,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
3,DETROIT MURAL #0690,FFTY,"8801 Oakland Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
4,DETROIT MURAL #0689,Laura Finlay,"4126 Third Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
5,DETROIT MURAL #0687,Ndubisi Okoye,"7900 Mack Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
6,DETROIT MURAL #0686,Michael Burdick and Kelly Golden,"10125 East Jefferson, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
7,DETROIT MURAL #0671,Bernal Perez,"6600 West Vernor Highway, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
8,DETROIT MURAL #0682,Tylonn J. Sawyer,"1550 Taylor Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
9,DETROIT MURAL #0625,Patch Whisky,"4847 Fort Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...


In [63]:
#drop duplicates and reset index (index may be used as primary key in database)
df = df.drop_duplicates().reset_index(drop=True)

In [71]:
#next page
page_link = "https://detroitmopa.org/artworks?offset=1509132420361&tag=Murals"
df = df.append(MuralScrape(page_link, df), ignore_index=True)
df= df.drop_duplicates().reset_index(drop=True)
df

,muralName,artistName,address,year,imgLink
0,DETROIT MURAL #0674,Pat Perry,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
1,DETROIT MURAL #0694,Tony Whlgn,"19031 Grand River Ave, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
2,DETROIT MURAL #0693,Ledania,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
3,DETROIT MURAL #0690,FFTY,"8801 Oakland Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
4,DETROIT MURAL #0689,Laura Finlay,"4126 Third Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
5,DETROIT MURAL #0687,Ndubisi Okoye,"7900 Mack Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
6,DETROIT MURAL #0686,Michael Burdick and Kelly Golden,"10125 East Jefferson, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
7,DETROIT MURAL #0671,Bernal Perez,"6600 West Vernor Highway, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
8,DETROIT MURAL #0682,Tylonn J. Sawyer,"1550 Taylor Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
9,DETROIT MURAL #0625,Patch Whisky,"4847 Fort Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...


In [84]:
page_link = "https://detroitmopa.org/artworks/tag/Eastern+Market+Murals"
df = df.append(MuralScrape(page_link, df), ignore_index=True)
df

['DETROIT MURAL #0698', 'DETROIT MURAL #0670', 'DETROIT MURAL #0677', 'DETROIT MURAL #0691', 'DETROIT MURAL #0284', 'DETROIT MURAL #0286', 'DETROIT MURAL #0285', 'DETROIT MURAL #0279', 'DETROIT MURAL #0278', 'DETROIT MURAL #0277', 'DETROIT MURAL #0280', 'DETROIT MURAL #0274', 'DETROIT MURAL #0276', 'DETROIT MURAL #0275', 'DETROIT MURAL #0272', 'DETROIT MURAL #0271', 'DETROIT MURAL #0273', 'DETROIT MURAL #0269', 'DETROIT MURAL #0270', 'DETROIT MURAL #0266']
['https://static1.squarespace.com/static/5ab0771496e76f89eeebf997/5ca3c3e7e4966ba9b7e6f488/5b572a6603ce642b5e2b1e35/1557315761343/FullSizeRender-2.jpg', 'https://static1.squarespace.com/static/5ab0771496e76f89eeebf997/5ca3c3e7e4966ba9b7e6f488/5b572a878a922dbc8509a20f/1557315761348/FullSizeRender.jpg', 'https://static1.squarespace.com/static/5ab0771496e76f89eeebf997/5ca3c3e7e4966ba9b7e6f488/5b572ac2562fa765b769039b/1557315761353/FullSizeRender-1.jpg', 'https://static1.squarespace.com/static/5ab0771496e76f89eeebf997/5ca3c3e7e4966ba9b7e

,muralName,artistName,address,year,imgLink
0,DETROIT MURAL #0674,Pat Perry,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
1,DETROIT MURAL #0694,Tony Whlgn,"19031 Grand River Ave, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
2,DETROIT MURAL #0693,Ledania,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
3,DETROIT MURAL #0690,FFTY,"8801 Oakland Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
4,DETROIT MURAL #0689,Laura Finlay,"4126 Third Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
5,DETROIT MURAL #0687,Ndubisi Okoye,"7900 Mack Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
6,DETROIT MURAL #0686,Michael Burdick and Kelly Golden,"10125 East Jefferson, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
7,DETROIT MURAL #0671,Bernal Perez,"6600 West Vernor Highway, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
8,DETROIT MURAL #0682,Tylonn J. Sawyer,"1550 Taylor Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
9,DETROIT MURAL #0625,Patch Whisky,"4847 Fort Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...


In [85]:
df = df.drop_duplicates()

In [87]:
page_link = "https://detroitmopa.org/artworks?offset=1506454620707&tag=Eastern+Market+Murals"
df = df.append(MuralScrape(page_link, df), ignore_index=True)
df

['DETROIT MURAL #0268', 'DETROIT MURAL #0262', 'DETROIT MURAL #0263', 'DETROIT MURAL #0257', 'DETROIT MURAL #0258', 'DETROIT MURAL #0256', 'DETROIT MURAL #0255', 'DETROIT MURAL #0234', 'DETROIT MURAL #0231', 'DETROIT MURAL #0232', 'DETROIT MURAL #0233', 'DETROIT MURAL #0230', 'DETROIT MURAL #0120', 'DETROIT MURAL #0119', 'DETROIT MURAL #0101', 'DETROIT MURAL #0186', 'DETROIT MURAL #0225', 'DETROIT MURAL #0035', 'DETROIT MURAL #0214', 'DETROIT MURAL #0226']
['https://static1.squarespace.com/static/5ab0771496e76f89eeebf997/5ca3c3e7e4966ba9b7e6f488/5ab90cb988251b9c242748a9/1559747478446/FullSizeRender.jpg', 'https://static1.squarespace.com/static/5ab0771496e76f89eeebf997/5ca3c3e7e4966ba9b7e6f488/5ab90bd903ce644cf63583ca/1559747510331/Jeremiah-Britton.JPG', 'https://static1.squarespace.com/static/5ab0771496e76f89eeebf997/5ca3c3e7e4966ba9b7e6f488/5ab90bfe03ce644cf6358bef/1559747532243/LunchMoney-WeWork-2017-Detroit-Mural-Project.jpg', 'https://static1.squarespace.com/static/5ab0771496e76f89

,muralName,artistName,address,year,imgLink
0,DETROIT MURAL #0674,Pat Perry,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
1,DETROIT MURAL #0694,Tony Whlgn,"19031 Grand River Ave, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
2,DETROIT MURAL #0693,Ledania,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
3,DETROIT MURAL #0690,FFTY,"8801 Oakland Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
4,DETROIT MURAL #0689,Laura Finlay,"4126 Third Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
5,DETROIT MURAL #0687,Ndubisi Okoye,"7900 Mack Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
6,DETROIT MURAL #0686,Michael Burdick and Kelly Golden,"10125 East Jefferson, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
7,DETROIT MURAL #0671,Bernal Perez,"6600 West Vernor Highway, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
8,DETROIT MURAL #0682,Tylonn J. Sawyer,"1550 Taylor Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
9,DETROIT MURAL #0625,Patch Whisky,"4847 Fort Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...


In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 5 columns):
muralName     180 non-null object
artistName    180 non-null object
address       180 non-null object
year          180 non-null object
imgLink       180 non-null object
dtypes: object(5)
memory usage: 7.1+ KB


In [97]:
page_link = "https://detroitmopa.org/artworks?offset=1474577460207&tag=Eastern+Market+Murals"
df = df.append(MuralScrape(page_link, df), ignore_index=True)
df

['DETROIT MURAL #0215', 'DETROIT MURAL #0192', 'DETROIT MURAL #0184', 'DETROIT MURAL #0182', 'DETROIT MURAL #0181', 'DETROIT MURAL #0180', 'DETROIT MURAL #0171', 'DETROIT MURAL #0170', 'DETROIT MURAL #0163', 'DETROIT MURAL #0161', 'DETROIT MURAL #0154', 'DETROIT MURAL #0152', 'DETROIT MURAL #0148', 'DETROIT MURAL #0144', 'DETROIT MURAL #0138', 'DETROIT MURAL #0115', 'DETROIT MURAL #0114', 'DETROIT MURAL #0109', 'DETROIT MURAL #0162', 'DETROIT MURAL #0227']
['https://static1.squarespace.com/static/5ab0771496e76f89eeebf997/5ca3c3e7e4966ba9b7e6f488/5ab1d0ab2b6a285eb10ed94e/1559748030461/tumblr_otogd5zRrM1w75qkio1_1280.jpg', 'https://static1.squarespace.com/static/5ab0771496e76f89eeebf997/5ca3c3e7e4966ba9b7e6f488/5ab1ccd62b6a285eb10e3bf6/1559748050428/tumblr_oto80uqrJt1w75qkio1_1280.jpg', 'https://static1.squarespace.com/static/5ab0771496e76f89eeebf997/5ca3c3e7e4966ba9b7e6f488/5ab1c8aa8a922d21ca7f27ed/1559748075328/1521600703475.jpeg', 'https://static1.squarespace.com/static/5ab0771496e76f

,muralName,artistName,address,year,imgLink
0,DETROIT MURAL #0674,Pat Perry,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
1,DETROIT MURAL #0694,Tony Whlgn,"19031 Grand River Ave, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
2,DETROIT MURAL #0693,Ledania,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
3,DETROIT MURAL #0690,FFTY,"8801 Oakland Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
4,DETROIT MURAL #0689,Laura Finlay,"4126 Third Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
5,DETROIT MURAL #0687,Ndubisi Okoye,"7900 Mack Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
6,DETROIT MURAL #0686,Michael Burdick and Kelly Golden,"10125 East Jefferson, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
7,DETROIT MURAL #0671,Bernal Perez,"6600 West Vernor Highway, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
8,DETROIT MURAL #0682,Tylonn J. Sawyer,"1550 Taylor Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
9,DETROIT MURAL #0625,Patch Whisky,"4847 Fort Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...


In [99]:
df= df.drop_duplicates()
df

,muralName,artistName,address,year,imgLink
0,DETROIT MURAL #0674,Pat Perry,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
1,DETROIT MURAL #0694,Tony Whlgn,"19031 Grand River Ave, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
2,DETROIT MURAL #0693,Ledania,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
3,DETROIT MURAL #0690,FFTY,"8801 Oakland Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
4,DETROIT MURAL #0689,Laura Finlay,"4126 Third Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
5,DETROIT MURAL #0687,Ndubisi Okoye,"7900 Mack Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
6,DETROIT MURAL #0686,Michael Burdick and Kelly Golden,"10125 East Jefferson, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
7,DETROIT MURAL #0671,Bernal Perez,"6600 West Vernor Highway, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
8,DETROIT MURAL #0682,Tylonn J. Sawyer,"1550 Taylor Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
9,DETROIT MURAL #0625,Patch Whisky,"4847 Fort Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...


In [100]:
df.reset_index(drop=True)

,muralName,artistName,address,year,imgLink
0,DETROIT MURAL #0674,Pat Perry,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
1,DETROIT MURAL #0694,Tony Whlgn,"19031 Grand River Ave, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
2,DETROIT MURAL #0693,Ledania,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
3,DETROIT MURAL #0690,FFTY,"8801 Oakland Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
4,DETROIT MURAL #0689,Laura Finlay,"4126 Third Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
5,DETROIT MURAL #0687,Ndubisi Okoye,"7900 Mack Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
6,DETROIT MURAL #0686,Michael Burdick and Kelly Golden,"10125 East Jefferson, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
7,DETROIT MURAL #0671,Bernal Perez,"6600 West Vernor Highway, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
8,DETROIT MURAL #0682,Tylonn J. Sawyer,"1550 Taylor Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
9,DETROIT MURAL #0625,Patch Whisky,"4847 Fort Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...


In [101]:
page_link = "https://detroitmopa.org/artworks?offset=1472763120116&tag=Eastern+Market+Murals"
df = df.append(MuralScrape(page_link, df), ignore_index=True)
df

['DETROIT MURAL #0155', 'DETROIT MURAL #0153', 'DETROIT MURAL #0149', 'DETROIT MURAL #0143', 'DETROIT MURAL #0139', 'DETROIT MURAL #0140', 'DETROIT MURAL #0137', 'DETROIT MURAL #0295', 'DETROIT MURAL #0293', 'DETROIT MURAL #0292', 'DETROIT MURAL #0291', 'DETROIT MURAL #0289', 'DETROIT MURAL #0267', 'DETROIT MURAL #0265', 'DETROIT MURAL #0259', 'DETROIT MURAL #0355', 'DETROIT MURAL #0406', 'DETROIT MURAL #0151', 'DETROIT MURAL #0580', 'DETROIT MURAL #0229']
['https://static1.squarespace.com/static/5ab0771496e76f89eeebf997/5ca3c3e7e4966ba9b7e6f488/5ab1ca86575d1f9478b3649c/1559748583673/Ghost-Beard-Patch-Whisky-2016-Detroit-Mural-Project.jpg', 'https://static1.squarespace.com/static/5ab0771496e76f89eeebf997/5ca3c3e7e4966ba9b7e6f488/5ab1ca9c88251b292be99993/1559748601445/Hueman-2016-Detroit-Mural-Project.jpg', 'https://static1.squarespace.com/static/5ab0771496e76f89eeebf997/5ca3c3e7e4966ba9b7e6f488/5ab1cbb988251b292be9c8f5/1559748614623/Detroit-Mural-Project-2016-Cey-Adams.jpg', 'https://s

,muralName,artistName,address,year,imgLink
0,DETROIT MURAL #0674,Pat Perry,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
1,DETROIT MURAL #0694,Tony Whlgn,"19031 Grand River Ave, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
2,DETROIT MURAL #0693,Ledania,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
3,DETROIT MURAL #0690,FFTY,"8801 Oakland Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
4,DETROIT MURAL #0689,Laura Finlay,"4126 Third Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
5,DETROIT MURAL #0687,Ndubisi Okoye,"7900 Mack Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
6,DETROIT MURAL #0686,Michael Burdick and Kelly Golden,"10125 East Jefferson, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
7,DETROIT MURAL #0671,Bernal Perez,"6600 West Vernor Highway, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
8,DETROIT MURAL #0682,Tylonn J. Sawyer,"1550 Taylor Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
9,DETROIT MURAL #0625,Patch Whisky,"4847 Fort Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...


In [102]:
df = df.drop_duplicates().reset_index(drop=True)

In [104]:
page_link = "https://detroitmopa.org/artworks?offset=1445547120498&tag=Eastern+Market+Murals"
df = df.append(MuralScrape(page_link, df), ignore_index=True)
df= df.drop_duplicates().reset_index(drop=True)
df

['DETROIT MURAL #0294', 'DETROIT MURAL #0287', 'DETROIT MURAL #0288', 'DETROIT MURAL #0187', 'DETROIT MURAL #0179', 'DETROIT MURAL #0169', 'DETROIT MURAL #0160', 'DETROIT MURAL #0158', 'DETROIT MURAL #0145', 'DETROIT MURAL #0121', 'DETROIT MURAL #0111', 'DETROIT MURAL #0097', 'DETROIT MURAL #0178', 'DETROIT MURAL #0228', 'DETROIT MURAL #0223', 'DETROIT MURAL #0213', 'DETROIT MURAL #0212', 'DETROIT MURAL #0191', 'DETROIT MURAL #0188', 'DETROIT MURAL #0159']
['https://static1.squarespace.com/static/5ab0771496e76f89eeebf997/5ca3c3e7e4966ba9b7e6f488/5ab941536d2a7301d3c983a7/1559749168712/FullSizeRender.jpg', 'https://static1.squarespace.com/static/5ab0771496e76f89eeebf997/5ca3c3e7e4966ba9b7e6f488/5ab93fc7758d46cb551df8b9/1559749191259/FullSizeRender-1.jpg', 'https://static1.squarespace.com/static/5ab0771496e76f89eeebf997/5ca3c3e7e4966ba9b7e6f488/5ab9400170a6ad454e5c8e32/1559749211151/FullSizeRender-2.jpg', 'https://static1.squarespace.com/static/5ab0771496e76f89eeebf997/5ca3c3e7e4966ba9b7e

,muralName,artistName,address,year,imgLink
0,DETROIT MURAL #0674,Pat Perry,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
1,DETROIT MURAL #0694,Tony Whlgn,"19031 Grand River Ave, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
2,DETROIT MURAL #0693,Ledania,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
3,DETROIT MURAL #0690,FFTY,"8801 Oakland Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
4,DETROIT MURAL #0689,Laura Finlay,"4126 Third Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
5,DETROIT MURAL #0687,Ndubisi Okoye,"7900 Mack Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
6,DETROIT MURAL #0686,Michael Burdick and Kelly Golden,"10125 East Jefferson, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
7,DETROIT MURAL #0671,Bernal Perez,"6600 West Vernor Highway, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
8,DETROIT MURAL #0682,Tylonn J. Sawyer,"1550 Taylor Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
9,DETROIT MURAL #0625,Patch Whisky,"4847 Fort Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...


In [105]:
page_link = "https://detroitmopa.org/artworks?offset=1441139160848&tag=Eastern+Market+Murals"
df = df.append(MuralScrape(page_link, df), ignore_index=True)
df= df.drop_duplicates().reset_index(drop=True)
df

['DETROIT MURAL #0156', 'DETROIT MURAL #0157', 'DETROIT MURAL #0150', 'DETROIT MURAL #0123', 'DETROIT MURAL #0124', 'DETROIT MURAL #0296', 'DETROIT MURAL #0290', 'DETROIT MURAL #0122', 'DETROIT MURAL #0118', 'DETROIT MURAL #0116', 'DETROIT MURAL #0112', 'DETROIT MURAL #0113', 'DETROIT MURAL #0264', 'DETROIT MURAL #0372', 'DETROIT MURAL #0371']
['https://static1.squarespace.com/static/5ab0771496e76f89eeebf997/5ca3c3e7e4966ba9b7e6f488/5ab1c9f42b6a285eb10dc302/1559750064114/Detroit-Mural-Sharktoof-2015.jpg', 'https://static1.squarespace.com/static/5ab0771496e76f89eeebf997/5ca3c3e7e4966ba9b7e6f488/5ab1c9db562fa7620d31acc0/1559749698813/Detroit-Mural-Ashew-Hannah-Stouffer.jpeg', 'https://static1.squarespace.com/static/5ab0771496e76f89eeebf997/5ca3c3e7e4966ba9b7e6f488/5ab1cac22b6a285eb10de66f/1559749715802/Detroit-Mural-Rone-West.jpg', 'https://static1.squarespace.com/static/5ab0771496e76f89eeebf997/5ca3c3e7e4966ba9b7e6f488/5ab1d266562fa7620d330d40/1559749731545/Detroit-Mural-Jarus.jpg', 'ht

,muralName,artistName,address,year,imgLink
0,DETROIT MURAL #0674,Pat Perry,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
1,DETROIT MURAL #0694,Tony Whlgn,"19031 Grand River Ave, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
2,DETROIT MURAL #0693,Ledania,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
3,DETROIT MURAL #0690,FFTY,"8801 Oakland Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
4,DETROIT MURAL #0689,Laura Finlay,"4126 Third Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
5,DETROIT MURAL #0687,Ndubisi Okoye,"7900 Mack Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
6,DETROIT MURAL #0686,Michael Burdick and Kelly Golden,"10125 East Jefferson, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
7,DETROIT MURAL #0671,Bernal Perez,"6600 West Vernor Highway, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
8,DETROIT MURAL #0682,Tylonn J. Sawyer,"1550 Taylor Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
9,DETROIT MURAL #0625,Patch Whisky,"4847 Fort Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...


In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 5 columns):
muralName     175 non-null object
artistName    175 non-null object
address       175 non-null object
year          175 non-null object
imgLink       175 non-null object
dtypes: object(5)
memory usage: 6.9+ KB


In [ ]:
#export to csv
df.to_csv("murals.csv")